In [ ]:
%pip install pandas
%pip install pyspark==3.5.0
%pip install -q findspark
%pip install py4j
%pip install pyarrow
%pip install numpy

In [1]:
# for local operations
import os
os.environ['JAVA_HOME'] = "C:\\Program Files\\Java\\jdk-11"
os.environ['HADOOP_HOME'] = "C:\\hadoop-3.3.6"

import pandas as pd
from pyspark.sql import functions as f

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .getOrCreate()
spark

In [3]:
from pyspark.sql import types as T

data_schema = T.StructType([
    T.StructField('contract_id', T.StringType(), False),
    T.StructField('timestamp', T.TimestampType(), False),
    T.StructField('value', T.DoubleType(), True),
    T.StructField('value_source', T.StringType(), False),
    T.StructField('annotations', T.StringType(), True)
])

df_raw_time_series = spark.read.format('json') \
    .schema(data_schema) \
    .load('project_data/project/raw_time_series/json/part-00000-3992c3d5-97c8-4e7d-8328-b7c8379135e8-c000.json')
df_raw_time_series.show()
df_raw_time_series.printSchema()

+--------------------+-------------------+-------------------+------------+--------------------+
|         contract_id|          timestamp|              value|value_source|         annotations|
+--------------------+-------------------+-------------------+------------+--------------------+
|  04 _02 _111 _CHR12|2023-01-01 03:15:00|0.01887007418980177| measurement|{"region":"Europe...|
|  04_02 _ 111 _CHR12|2023-01-01 16:15:00|           1.266225| measurement|{"region":"Europe...|
|  04_02_ 111 _CHR12 |2023-01-01 23:45:27|               NULL| measurement|{"region":"Europe...|
|  04 _02 _111 _CHR12|2023-01-02 03:30:00|0.02093419915470171| measurement|{"region":"Europe...|
|   04 _02_111_CHR12 |2023-01-02 09:45:00| 0.0142526590154003| measurement|{"region":"Europe...|
|  04_02 _ 111 _CHR12|2023-01-02 18:45:00|0.07874463371047623| measurement|{"region":"Europe...|
|   04 _02_111_CHR12 |2023-01-03 00:45:00| 0.1950477376343327| measurement|{"region":"Europe...|
|  04_02_ 111 _CHR12 |2023-01-

Data clean-up progress - if ruled more than once, all values will be turned to NULL

In [4]:
df_raw_time_series = (
    df_raw_time_series
    .withColumn('contract_id', f.regexp_replace(f.col('contract_id'), ' ', ''))
    .withColumn('value_source', f.when(f.col('value').isNull(), 'missing').otherwise(f.col('value_source')))
    .withColumn('timestamp',
                f.from_unixtime(f.round(f.unix_timestamp('timestamp') / (15 * 60)) * (15 * 60)))
    .withColumn('timestamp',f.date_format('timestamp', 'yyyy-MM-dd HH:mm'))
)
df_raw_time_series.show()

+---------------+----------------+-------------------+------------+--------------------+
|    contract_id|       timestamp|              value|value_source|         annotations|
+---------------+----------------+-------------------+------------+--------------------+
|04_02_111_CHR12|2023-01-01 03:15|0.01887007418980177| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 16:15|           1.266225| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-01 23:45|               NULL|     missing|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 03:30|0.02093419915470171| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 09:45| 0.0142526590154003| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-02 18:45|0.07874463371047623| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 00:45| 0.1950477376343327| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023-01-03 01:00| 0.1494432740331963| measurement|{"region":"Europe...|
|04_02_111_CHR12|2023

Extracting a new "region" column from "annotations"

In [5]:
json_schema = T.StructType([
    T.StructField('region', T.StringType(), False)
])

df_raw_time_series = (
    df_raw_time_series
    .withColumn('region', f.from_json(f.col('annotations'), json_schema).getField('region'))
)
df_raw_time_series.show()

# check-up to see other regions
df_raw_time_series.select('region').distinct().show()

+---------------+----------------+-------------------+------------+--------------------+-------------+
|    contract_id|       timestamp|              value|value_source|         annotations|       region|
+---------------+----------------+-------------------+------------+--------------------+-------------+
|04_02_111_CHR12|2023-01-01 03:15|0.01887007418980177| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 16:15|           1.266225| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 23:45|               NULL|     missing|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-02 03:30|0.02093419915470171| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-02 09:45| 0.0142526590154003| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-02 18:45|0.07874463371047623| measurement|{"region":"Europe...|Europe/Berlin|
|04_02_111_CHR12|2023-01-03 00:45| 0.1950477376343327| measurement|{"regi

Customers with invalid regions will be removed from the database and saved on disk in a separate location.
- Suppose the regions respect the following format: `continent/city`

In [ ]:
df_invalid_region = (
    df_raw_time_series
    .filter((f.col('region').isNull()) |
            (f.trim(f.col('region')) == '') |
            (f.regexp_extract(f.col('region'), r'^[A-Za-z]+/[A-Za-z]+$', 0) == ''))
)

df_invalid_region.show()
# df_invalid_region.write.json('invalid_regions', mode="overwrite") - doesn't work locally, will update to new environment soon

+---------------+----------------+--------------------+------------+--------------------+--------+
|    contract_id|       timestamp|               value|value_source|         annotations|  region|
+---------------+----------------+--------------------+------------+--------------------+--------+
|01_02_155_CHR98|2023-01-01 05:00|  0.1407119333744049| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 07:00| 0.04566259682178497| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 07:30| 0.10312729328870773| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 08:30| 0.06968000531196594| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 09:30| 0.11096686869859695| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 09:45|   0.241357684135437| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155_CHR98|2023-01-01 12:30| 0.07323921471834183| measurement|{"region":"WakaWa...|WakaWaka|
|01_02_155

Py4JJavaError: An error occurred while calling o81.json.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOu

Extract date from the `timestamp` column in a new `utc_date` column

In [7]:
df_raw_time_series = (
    df_raw_time_series
    .withColumn('utc_date',f.date_format('timestamp', 'yyyy-MM-dd'))
)
df_raw_time_series.show()

+---------------+----------------+-------------------+------------+--------------------+-------------+----------+
|    contract_id|       timestamp|              value|value_source|         annotations|       region|  utc_date|
+---------------+----------------+-------------------+------------+--------------------+-------------+----------+
|04_02_111_CHR12|2023-01-01 03:15|0.01887007418980177| measurement|{"region":"Europe...|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-01 16:15|           1.266225| measurement|{"region":"Europe...|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-01 23:45|               NULL|     missing|{"region":"Europe...|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-02 03:30|0.02093419915470171| measurement|{"region":"Europe...|Europe/Berlin|2023-01-02|
|04_02_111_CHR12|2023-01-02 09:45| 0.0142526590154003| measurement|{"region":"Europe...|Europe/Berlin|2023-01-02|
|04_02_111_CHR12|2023-01-02 18:45|0.07874463371047623| measurement|{"region":"Europe...|

Calculate local date for the date and time of `timestamp`, based on the region, in a new `local_timestamp` column

In [8]:
df_raw_time_series = df_raw_time_series.withColumn(
    "local_timestamp",
    f.from_utc_timestamp(df_raw_time_series["timestamp"], df_raw_time_series["region"])
)

df_raw_time_series.show()

+---------------+----------------+-------------------+------------+--------------------+-------------+----------+-------------------+
|    contract_id|       timestamp|              value|value_source|         annotations|       region|  utc_date|    local_timestamp|
+---------------+----------------+-------------------+------------+--------------------+-------------+----------+-------------------+
|04_02_111_CHR12|2023-01-01 03:15|0.01887007418980177| measurement|{"region":"Europe...|Europe/Berlin|2023-01-01|2023-01-01 04:15:00|
|04_02_111_CHR12|2023-01-01 16:15|           1.266225| measurement|{"region":"Europe...|Europe/Berlin|2023-01-01|2023-01-01 17:15:00|
|04_02_111_CHR12|2023-01-01 23:45|               NULL|     missing|{"region":"Europe...|Europe/Berlin|2023-01-01|2023-01-02 00:45:00|
|04_02_111_CHR12|2023-01-02 03:30|0.02093419915470171| measurement|{"region":"Europe...|Europe/Berlin|2023-01-02|2023-01-02 04:30:00|
|04_02_111_CHR12|2023-01-02 09:45| 0.0142526590154003| measure

Extract date from the `local_timestamp` column in a new `local_date` column

In [9]:
df_raw_time_series = df_raw_time_series.withColumn(
    "local_date",
    f.to_date(df_raw_time_series["local_timestamp"])
)

df_raw_time_series.show()

+---------------+----------------+-------------------+------------+--------------------+-------------+----------+-------------------+----------+
|    contract_id|       timestamp|              value|value_source|         annotations|       region|  utc_date|    local_timestamp|local_date|
+---------------+----------------+-------------------+------------+--------------------+-------------+----------+-------------------+----------+
|04_02_111_CHR12|2023-01-01 03:15|0.01887007418980177| measurement|{"region":"Europe...|Europe/Berlin|2023-01-01|2023-01-01 04:15:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 16:15|           1.266225| measurement|{"region":"Europe...|Europe/Berlin|2023-01-01|2023-01-01 17:15:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 23:45|               NULL|     missing|{"region":"Europe...|Europe/Berlin|2023-01-01|2023-01-02 00:45:00|2023-01-02|
|04_02_111_CHR12|2023-01-02 03:30|0.02093419915470171| measurement|{"region":"Europe...|Europe/Berlin|2023-01-02|2023-01-02 04:30: